# **Lab Assignment 2 - Principal Component Analysis (PCA) and Neural Networks**

Go through each link provided and implement the tasks below. 

Make a short report (pdf format) that includes: (1) plots/diagrams generated during training and evaluation, (2) key console output such as model summaries and training logs (loss/accuracy per epoch). Since the labs are only graded Pass/Fail, the report can be very simple, just copy and paste your results and figures given when running your program.

Upload your code as .py file/s.

During the scheduled lab sessions, present your code and report to a teacher. The focus should be on your observations, interpreting the results (what changed, why it matters). 


## **Tasks**

### **Principal Component Analysis (PCA)**

1. Load or create a dataset with more than 2 dimensions.

2. Find the first 2 principal components **with and without using sklearn**.

3. Practice using PCA to preserve a certain percentage of variance.

4. Train a classification or regression neural network by using:
   - a) The original dataset  
   - b) Principal components

5. Repeat part 4 using **Kernel PCA** (linear, sigmoid, RBF).

6. Build a pipeline to tune the hyperparameters of Kernel PCA and also the neural network.  
   **Questions:** Which hyperparameters can be tuned?

---

**Reference / Sample Code:**  
Hands-On Machine Learning 2nd Edition, Chapter 8, Dimensionality Reduction:  
[PCA and Dimensionality Reduction Notebook on GitHub](https://github.com/ageron/handson-ml2/blob/master/08_dimensionality_reduction.ipynb)


In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Classification Problem Assignment

1. Load **IRIS** and **Fashion MNIST** datasets from Keras.  
2. Build and train a neural network to classify each of the loaded labeled datasets.  
3. Tune hyperparameters (e.g. number of hidden layers/units, activation functions). What else can you tune?  
4. Plot the loss and accuracy for both training and testing datasets.  
5. Save the weights of the model’s layers and use callbacks during training.  
6. Practice saving and loading the trained model.

---

**Reference / Sample Code:**  
Neural networks with Keras — classification & regression examples:  
[10_neural_nets_with_keras.ipynb on GitHub](https://github.com/ageron/handson-ml2/blob/master/10_neural_nets_with_keras.ipynb)


## Regression Problem Assignment

1. Load the **California Housing** dataset and split it into training and testing sets.  
2. Build and train a neural network to predict housing prices. Tune hyperparameters and discuss the results.  
3. Plot the training history (loss over epochs, validation loss, etc.).  
4. Change the learning rate, retrain the network, plot the network history again, and discuss how the learning rate affected performance.  
5. Save the weights of the model’s layers and use callbacks during training.  
6. Practice saving and loading the trained model.

---

**Reference / Sample Code:**  
Neural networks with Keras — classification & regression examples:  
[10_neural_nets_with_keras.ipynb on GitHub](https://github.com/ageron/handson-ml2/blob/master/10_neural_nets_with_keras.ipynb)
